In [1]:
from torch_geometric.data import HeteroData 
from collections import defaultdict 
import torch 

data = HeteroData()

# node_types = {0: 'paper', 1, 'author', ...}
# edge_types = {0: ('paper', 'cite', 'paper'), ...}
if True:
    with open('/mnt1/genghao/dataset/HGB/raw/Freebase/info.dat', 'r') as f:  # `info.dat`
        info = f.read().split('\n')
    start = info.index('TYPE\tMEANING') + 1
    end = info[start:].index('')
    n_types = [v.split('\t\t') for v in info[start:start + end]]
    n_types = {int(k): v.lower() for k, v in n_types}

    e_types = {}
    start = info.index('LINK\tSTART\tEND\tMEANING') + 1
    end = info[start:].index('')
    for key, row in enumerate(info[start:start + end]):
        row = row.split('\t')[1:]
        src, dst, rel = [v for v in row if v != '']
        src, dst = n_types[int(src)], n_types[int(dst)]
        rel = rel.split('-')[1]
        e_types[key] = (src, rel, dst)

# Extract node information:
mapping_dict = {}  # Maps global node indices to local ones.
x_dict = defaultdict(list)
num_nodes_dict = defaultdict(lambda: 0)
with open('/mnt1/genghao/dataset/HGB/raw/Freebase/node.dat', 'r') as f:  # `node.dat`
    xs = [v.split('\t') for v in f.read().split('\n')[:-1]]
for x in xs:
    n_id, n_type = int(x[0]), n_types[int(x[2])]
    mapping_dict[n_id] = num_nodes_dict[n_type]
    num_nodes_dict[n_type] += 1
    if len(x) >= 4:  # Extract features (in case they are given).
        x_dict[n_type].append([float(v) for v in x[3].split(',')])
for n_type in n_types.values():
    if len(x_dict[n_type]) == 0:
        data[n_type].num_nodes = num_nodes_dict[n_type]
    else:
        data[n_type].x = torch.tensor(x_dict[n_type])

edge_index_dict = defaultdict(list)
edge_weight_dict = defaultdict(list)
with open('/mnt1/genghao/dataset/HGB/raw/Freebase/link.dat', 'r') as f:  # `link.dat`
    edges = [v.split('\t') for v in f.read().split('\n')[:-1]]
for src, dst, rel, weight in edges:
    e_type = e_types[int(rel)]
    src, dst = mapping_dict[int(src)], mapping_dict[int(dst)]
    edge_index_dict[e_type].append([src, dst])
    edge_weight_dict[e_type].append(float(weight))
for e_type in e_types.values():
    edge_index = torch.tensor(edge_index_dict[e_type])
    edge_weight = torch.tensor(edge_weight_dict[e_type])
    data[e_type].edge_index = edge_index.t().contiguous()
    # Only add "weighted" edgel to the graph:
    if not torch.allclose(edge_weight, torch.ones_like(edge_weight)):
        data[e_type].edge_weight = edge_weight

# Node classification:
if True:
    with open('/mnt1/genghao/dataset/HGB/raw/Freebase/label.dat', 'r') as f:  # `label.dat`
        train_ys = [v.split('\t') for v in f.read().split('\n')[:-1]]
    with open('/mnt1/genghao/dataset/HGB/raw/Freebase/label.dat.test', 'r') as f:  # `label.dat.test`
        test_ys = [v.split('\t') for v in f.read().split('\n')[:-1]]
    for y in train_ys:
        n_id, n_type = mapping_dict[int(y[0])], n_types[int(y[2])]

        if not hasattr(data[n_type], 'y'):
            num_nodes = data[n_type].num_nodes
            if False:  # multi-label
                data[n_type].y = torch.zeros((num_nodes, num_classes))
            else:
                data[n_type].y = torch.full((num_nodes, ), -1).long()
            data[n_type].train_mask = torch.zeros(num_nodes).bool()
            data[n_type].test_mask = torch.zeros(num_nodes).bool()

        if False:  # multi-label
            for v in y[3].split(','):
                data[n_type].y[n_id, int(v)] = 1
        else:
            data[n_type].y[n_id] = int(y[3])
        data[n_type].train_mask[n_id] = True
        
    print(len(test_ys))
        
    for y in test_ys:
        n_id, n_type = mapping_dict[int(y[0])], n_types[int(y[2])]
        if False:  # multi-label
            for v in y[3].split(','):
                data[n_type].y[n_id, int(v)] = 1
        else:
            data[n_type].y[n_id] = int(y[3])
        data[n_type].test_mask[n_id] = True
        
data 

5568


HeteroData(
  book={
    num_nodes=40402,
    y=[40402],
    train_mask=[40402],
    test_mask=[40402]
  },
  film={ num_nodes=19427 },
  music={ num_nodes=82351 },
  sports={ num_nodes=1025 },
  people={ num_nodes=17641 },
  location={ num_nodes=9368 },
  organization={ num_nodes=2731 },
  business={ num_nodes=7153 },
  (book, and, book)={ edge_index=[2, 202674] },
  (book, to, film)={ edge_index=[2, 38299] },
  (book, on, sports)={ edge_index=[2, 6615] },
  (book, on, location)={ edge_index=[2, 26921] },
  (book, about, organization)={ edge_index=[2, 21900] },
  (film, and, film)={ edge_index=[2, 87838] },
  (music, in, book)={ edge_index=[2, 31486] },
  (music, in, film)={ edge_index=[2, 11291] },
  (music, and, music)={ edge_index=[2, 283670] },
  (music, for, sports)={ edge_index=[2, 8975] },
  (music, on, location)={ edge_index=[2, 42915] },
  (sports, in, film)={ edge_index=[2, 6763] },
  (sports, and, sports)={ edge_index=[2, 1290] },
  (sports, on, location)={ edge_index=[2, 6

In [2]:
y = data.y_dict['book'] 

(y > 0).float().mean() 

tensor(0.1863)

In [3]:
train_mask = data.train_mask_dict['book']
test_mask = data.test_mask_dict['book'] 
assert (train_mask & test_mask).float().sum() == 0 

train_mask.float().mean(), test_mask.float().mean() 

(tensor(0.0591), tensor(0.1378))

In [4]:
y.max() + 1 

tensor(7)

In [5]:
data.num_nodes_dict 

{'book': 40402,
 'film': 19427,
 'music': 82351,
 'sports': 1025,
 'people': 17641,
 'location': 9368,
 'organization': 2731,
 'business': 7153}

In [6]:
import pickle 

with open('/mnt1/genghao/dataset/HGB/processed/Freebase_hg.dict.pkl', 'wb') as fp: 
    pickle.dump(
        dict(
            num_nodes_dict = dict(data.num_nodes_dict), 
            edge_index_dict = dict(data.edge_index_dict),
            book_label = data.y_dict['book'], 
            book_train_mask = data.train_mask_dict['book'], 
            book_test_mask = data.test_mask_dict['book'], 
        ), 
        fp, 
    )